# Simple Hartree-Fock equation


* Calculate 16 nucleons s.p. energies by using H.F. theory. 

* Use H.O. basis 

* TBME is from N3LO EM potential after SRG

* However, the Hamiltonian contains external trapping Harmonic oscillator potential. 

* Thus, one can not simply get the binding energy of the system by computing H.F. energy. 

* To get ground state binding energy, one have to subtract additional center of mass motion in trapping potential. 


In [2]:
import numpy as np 
from decimal import Decimal
# expectation value for the one body part, Harmonic oscillator in three dimensions
def onebody(i, n, l):
        homega = 10.0
        return homega*(2*n[i] + l[i] + 1.5)

* Read s.p. states info from the file as following format 
(It is list of H.O. eigen states.) 

      i     n   l   2j  mj tz 
      1     0   0   1  -1  -1
      2     0   0   1   1  -1
      3     0   0   1  -1   1
      4     0   0   1   1   1
      ... 
     
* For example, there are 4 nucleons in (nlj)=(0,0,1/2) state     
* Total number of s.p. states are 80.      
* one body matrix element < i | h| i> = (2n+l+3/2)homega with homega=10 MeV.
  In other words, it is a h.o. Hamiltonian.  

In [3]:
Nparticles = 16
""" Read quantum numbers from file """
index = []
n = []
l = []
j = []
mj = []
tz = []
spOrbitals = 0
with open("nucleispnumbers.dat", "r") as qnumfile:
    for line in qnumfile:
        nums = line.split()
        if len(nums) != 0:
                index.append(int(nums[0]))
                n.append(int(nums[1]))
                l.append(int(nums[2]))
                j.append(int(nums[3]))
                mj.append(int(nums[4]))
                tz.append(int(nums[5]))
                spOrbitals += 1

* Two-body matrix elements are read from file as following structure 

     a  b  c  d      < ab| v| cd > 
     
     41 42 41 42     -.153495E+00
     
     41 42 41 57     0.552994E+00
     
     41 42 42 41     0.153495E+00
     
     41 42 57 41     -.552994E+00
     
     41 57 41 42     0.552994E+00
     
     41 57 41 57     -.128229E+01
     
     ....
     
* The interaction is from  N3LO interaction of Machleidt and Entem using the Similarity Renormalization Group approach method to renormalize the interaction, using an oscillator energy ℏω=10 MeV.

* <a b|v|c d> are listed for all possible combinations of non-zero matrix elements here. 
  ( Usually, one stores only part of matrix elements and use symmetry to get related matrix elements.)
   

In [4]:
""" Read two-nucleon interaction elements (integrals) from file, brute force 4-dim array """
nninteraction = np.zeros([spOrbitals, spOrbitals, spOrbitals, spOrbitals])
with open("nucleitwobody.dat", "r") as infile:
    for line in infile:
        number = line.split()
        a = int(number[0]) - 1
        b = int(number[1]) - 1
        c = int(number[2]) - 1
        d = int(number[3]) - 1
        nninteraction[a][b][c][d] = Decimal(number[4])

* singleparticleH = (e0,e1,e2....) array of s.p. energy for H_{HO} 

In [5]:
""" Set up single-particle integral """
singleparticleH = np.zeros(spOrbitals)
for i in range(spOrbitals):
    singleparticleH[i] = Decimal(onebody(i, n, l))

* C_{ab} is initialized as delta_{ab} 

* DensityMatrix_{ab} = sum_{i=1}^A C_{ai} C_{bi} =  delta_{a<F,b<F}  
    

In [6]:
""" Start HF-iterations, preparing variables and density matrix """

""" Coefficients for setting up density matrix, assuming only one along the diagonals """
C = np.eye(spOrbitals) # HF coefficients
DensityMatrix = np.zeros([spOrbitals,spOrbitals])
for gamma in range(spOrbitals):
    for delta in range(spOrbitals):
        sum = 0.0
        for i in range(Nparticles):
            sum += C[gamma][i]*C[delta][i]
        DensityMatrix[gamma][delta] = Decimal(sum)

* Here index $\alpha,\beta$ is for H.O. basis. While $i$ is for H.F. basis. 
* For H.O. basis $<\alpha|h_0|\beta> = e_\alpha \delta_{\alpha\beta}$
* TBME is from given $<\alpha\beta|v|\gamma\delta>$
* Construct Fork term matrix elements $F_{\alpha\beta}= h^{HF}_{\alpha\beta}$.
  (Note that Fock term depends on the density matrix or ReferenceState) 

* $h_{\alpha\beta}^{HF}=\langle \alpha | \hat{t}+\hat{u} | \beta \rangle+\sum_{j=1}^A\sum_{\gamma\delta} C^*_{j\gamma}C_{j\delta}\langle \alpha\gamma|\hat{v}|\beta\delta\rangle_{AS},$

* Solve $\sum_{\beta}\hat{h}_{\alpha\beta}^{HF}C_{i\beta}=\epsilon_i^{\mathrm{HF}}C_{i\alpha},$
* Iteration goes 
        (1) Construct Fock term from previous density matrix 
        (2) Solve Eigenvalue problem $\sum_{\beta}\hat{h}_{\alpha\beta}^{HF}C_{i\beta}=\epsilon_i^{\mathrm{HF}}C_{i\alpha},$
        (3) Compare difference= Mean squared Error of single particle levels (eigen values) between iteration
        (4) Iterate until difference becomes small.

* Once s.p. levels converges, we have HF basis such that $< i|F|j>=e_i \delta_{ij}$, $|i> = C_{i\alpha}|\alpha>$.

* New reference is a occupied states $i<=A$. New density matrix in this basis is of cource, 
  
  $\rho_{ij}= \sum_\alpha C^*_{i\alpha} C_{j\alpha}=\delta_{ij}\delta_{i<=A}$. 

* The Hamiltonian used in this problem includes C.M. motion in H.O.  
 
  $ H=\sum_i  (T_i + U^{ho}_i) +\sum_{ij} v_{ij}$
       
* The expectation value of energy becomes 

  $< H > = <HF|h_0|HF> + 1/4 <HF|V|HF> = \sum_{i<=A} e_i +\frac{1}{2}\sum_{ij<=A} < ij|v|ij> 
       = \sum_{i<=A} e_i +\frac{1}{2}\sum_{\alpha\beta} \rho_{\alpha\gamma}\rho_{\beta\delta}< \alpha\beta|v|\alpha\beta> $
  
* The code updates density matrices, $\rho_{\alpha,\beta} = < HF|c^\dagger_\alpha c_\beta|HF> = \sum_{i} C^*_{i\alpha}C_{i\beta}$ , 
  instead of storing coefficients $C_{i\beta}$. 
  
* __This code is not for finite nuclei. Rather it is for neutron drop in Harmonic oscillator trap.__   

## Main HF routine

In [7]:
        maxHFiter = 100
        epsilon =  1.0e-5 
        difference = 1.0
        hf_count = 0
        oldenergies = np.zeros(spOrbitals)
        newenergies = np.zeros(spOrbitals)
        while hf_count < maxHFiter and difference > epsilon:
                print("############### Iteration %i ###############" % hf_count)
                HFmatrix = np.zeros([spOrbitals,spOrbitals])            
                for alpha in range(spOrbitals):
                        for beta in range(spOrbitals):
                            """  If tests for three-dimensional systems, including isospin conservation """
                            if l[alpha] != l[beta] and j[alpha] != j[beta] and mj[alpha] != mj[beta] and tz[alpha] != tz[beta]: continue
                            """  Setting up the Fock matrix using the density matrix and antisymmetrized NN interaction in m-scheme """
                            sumFockTerm = 0.0
                            for gamma in range(spOrbitals):
                                for delta in range(spOrbitals):
                                    if (mj[alpha]+mj[gamma]) != (mj[beta]+mj[delta]) and (tz[alpha]+tz[gamma]) != (tz[beta]+tz[delta]): continue
                                    sumFockTerm += DensityMatrix[gamma][delta]*nninteraction[alpha][gamma][beta][delta]
                            HFmatrix[alpha][beta] = Decimal(sumFockTerm)
                            """  Adding the one-body term, here plain harmonic oscillator """
                            if beta == alpha:   HFmatrix[alpha][alpha] += singleparticleH[alpha]
                spenergies, C = np.linalg.eigh(HFmatrix)
                """ Setting up new density matrix in m-scheme """
                DensityMatrix = np.zeros([spOrbitals,spOrbitals])
                for gamma in range(spOrbitals):
                    for delta in range(spOrbitals):
                        sum = 0.0
                        for i in range(Nparticles):
                            sum += C[gamma][i]*C[delta][i]
                        DensityMatrix[gamma][delta] = Decimal(sum)
                newenergies = spenergies
                """ Brute force computation of difference between previous and new sp HF energies """
                sum =0.0
                for i in range(spOrbitals):
                    sum += (abs(newenergies[i]-oldenergies[i]))/spOrbitals
                difference = sum
                oldenergies = newenergies
                print ("Single-particle energies, ordering may have changed ")
                for i in range(spOrbitals):
                    print('{0:4d}  {1:.4f}'.format(i, Decimal(oldenergies[i])))
                hf_count += 1

############### Iteration 0 ###############
Single-particle energies, ordering may have changed 
   0  -23.6381
   1  -23.6381
   2  -23.5230
   3  -23.5230
   4  -3.3691
   5  -3.3691
   6  -3.3690
   7  -3.3690
   8  -3.2777
   9  -3.2777
  10  -3.2777
  11  -3.2777
  12  -1.2622
  13  -1.2622
  14  -1.1717
  15  -1.1717
  16  18.8137
  17  18.8137
  18  18.8137
  19  18.8137
  20  18.8137
  21  18.8137
  22  18.8602
  23  18.8602
  24  18.8602
  25  18.8602
  26  18.8602
  27  18.8602
  28  20.7992
  29  20.7992
  30  20.8400
  31  20.8400
  32  21.4603
  33  21.4603
  34  21.4603
  35  21.4603
  36  21.5058
  37  21.5058
  38  21.5058
  39  21.5058
  40  34.7143
  41  34.7143
  42  34.7143
  43  34.7143
  44  34.7143
  45  34.7143
  46  34.7144
  47  34.7144
  48  34.7436
  49  34.7436
  50  34.7436
  51  34.7436
  52  34.7436
  53  34.7436
  54  34.7436
  55  34.7436
  56  34.9734
  57  34.9734
  58  34.9734
  59  34.9734
  60  35.0012
  61  35.0012
  62  35.0012
  63  35.0012
  6

## Test HF energy relations

Compute HF energy 
$
  E[\Psi] 
  = \sum_{i=1}^A \sum_{\alpha\beta} C^*_{i\alpha}C_{i\beta}\langle \alpha | h_{ho} | \beta \rangle +
  \frac{1}{2}\sum_{ij=1}^A\sum_{{\alpha\beta\gamma\delta}} C^*_{i\alpha}C^*_{j\beta}C_{i\gamma}C_{j\delta}\langle \alpha\beta|\hat{v}|\gamma\delta\rangle_{AS}. \label{FunctionalEPhi3}
$

* sum1 $=\sum_{i=1}^A \langle i|h_{ho}|i\rangle =\sum_{i=1}^A \sum_{\alpha\beta} C^*_{i\alpha}C_{i\beta}\langle \alpha | h_{ho} | \beta \rangle$

* sum2 $=\sum_{ij=1}^A \langle ij |\hat{v}|ij \rangle_{AS} = \sum_{ij=1}^A\sum_{{\alpha\beta\gamma\delta}} C^*_{i\alpha}C^*_{j\beta}C_{i\gamma}C_{j\delta}\langle \alpha\beta|\hat{v}|\gamma\delta\rangle_{AS}$

* sum of HF s.p.energy $\sum_{i} e_i^{HF} =\sum_{ij} f_{ij} \delta_{ij}=$ sum1+sum2  

* Note that HF energy is not simply the sum of HF s.p. energies. 
  
  $E_{HF} = \sum_{i} e_i^{HF} -\frac{1}{2} \sum_{ij} \langle ij|v|ij\rangle = sum1+sum2 -0.5 * sum2 = sum1+0.5*sum2$
    
* How one can remove c.m. energy and H.O. potential contribution?
  The above Hamiltonian, corresponds to adding C.M. H.O. term $P^2/{2mA}+\hbar/2 mA* \Omega^2$
  which would have $\hbar* \Omega *3/2$ at lowest level.

In [8]:
sum1 = 0.0
for alpha in range(spOrbitals):
    sum1 += DensityMatrix[alpha][alpha]*singleparticleH[alpha]
sum2 = 0.0     
for alpha in range(spOrbitals):   
    for beta in range(spOrbitals):   
        for gamma in range(spOrbitals):   
            for delta in range(spOrbitals):   
                sum2 += DensityMatrix[alpha][gamma]*DensityMatrix[beta][delta]*nninteraction[alpha][beta][gamma][delta]

* Check above relation between s.p. HF energy and sum1 and sum2

16O nndc BE/A=7976.206 
Thus, BE(16O) = -7.976206*16 = -127.6 MeV 

In [10]:
# test relation  $\sum_{i} e_i^{HF} =$ sum1+sum2
print('HF s.p. energies')
print(np.sum(spenergies[0:16]), sum1, sum2, sum1+sum2)  

# H.F. energy 
print('E_HF   E(16O)')
print(np.sum(spenergies[0:16])-0.5*sum2-10*3/2.,-7.976206*16 )  

HF s.p. energies
-282.5477063774011 408.09016653972793 -690.6379254472881 -282.5477589075602
E_HF
47.77125634624298 -127.619296


## HF calculation for pairing model 

$ h^{HF}_{\alpha\beta} = <\alpha|h_0|\beta> +\sum_{j<=F} \sum_{\gamma\delta} C^*_{j\gamma}C_{j\delta}<\alpha\gamma|v|\beta\delta> 
                       = <\alpha|h_0|\beta> +\sum_{\gamma\delta} \rho_{\gamma\delta} <\alpha\gamma|v|\beta\delta> $                                       